### All Imports

In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

### Face Section

In [ ]:
# Define constants
data_dir = "Dataset/VISA_Face/VISA_Face"
input_shape = (128, 128, 3)  # Input image dimensions
num_classes = 25
batch_size = 32
epochs = 1
embedding_dim = 64  # Dimensionality of the embedding space

# Function to load and preprocess images


def preprocess_image(image_path):
    try:
        img = load_img(image_path, target_size=input_shape[:2])
        img = img_to_array(img) / 255.0  # Normalize pixel values
        return img
    except Exception as e:
        print(f"Error loading image: {image_path}. {e}")
        return None


# Function to recursively search for image files in directories


def find_images(directory):
    images = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                images.append(os.path.join(root, file))
    return images


# Load and preprocess dataset
class_dirs = os.listdir(data_dir)
class_images = {class_dir: find_images(os.path.join(
    data_dir, class_dir)) for class_dir in class_dirs}

pairs = []
labels = []

for class_dir, images in class_images.items():
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            pairs.append((images[i], images[j]))
            labels.append(1 if class_dir == class_images else 0)

# Split dataset into train and test sets
pairs_train, pairs_test, labels_train, labels_test = train_test_split(
    pairs, labels, test_size=0.2, random_state=42)

# Siamese network architecture


def create_siamese_network(input_shape, embedding_dim):
    input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(embedding_dim)(x)
    return models.Model(input, output)


# Create Siamese network branches
base_network = create_siamese_network(input_shape, embedding_dim)
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = tf.abs(processed_a - processed_b)

# Output layer
output = layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Create a function to generate batches of pairs with GPU acceleration


# Function to generate a single batch of pairs
def generate_single_batch(pairs, labels, batch_indices):
    X1 = np.zeros((len(batch_indices), *input_shape))
    X2 = np.zeros((len(batch_indices), *input_shape))
    Y = np.zeros(len(batch_indices))

    for i, idx in enumerate(batch_indices):
        pair = pairs[idx]
        label = labels[idx]
        img1 = preprocess_image(pair[0])
        img2 = preprocess_image(pair[1])
        X1[i] = img1
        X2[i] = img2
        Y[i] = label

    return [X1, X2], Y


def generate_batch(pairs, labels, batch_size):
    while True:
        batch_indices = random.sample(range(len(pairs)), batch_size)
        yield generate_single_batch(pairs, labels, batch_indices)


# Train the model
history = model.fit(
    generate_batch(pairs_train, labels_train, batch_size),
    steps_per_epoch=len(labels_train) // batch_size,
    epochs=epochs,
    validation_data=generate_batch(pairs_test, labels_test, batch_size),
    validation_steps=len(labels_test) // batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    generate_batch(pairs_test, labels_test, batch_size),
    steps=len(labels_test) // batch_size
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")